## 套件匯入

In [13]:
from selenium import webdriver
from time import sleep
import pandas as pd

## 啟動瀏覽器（https://law.judicial.gov.tw/FJUD/default.aspx )

In [8]:
# 司法院官網
link = 'https://law.judicial.gov.tw/FJUD/default.aspx'
options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.get(link)

# 定位輸入框，寫入搜尋 <關鍵字>
TEXT = '證券詐欺'
sleep(2)
driver.find_element_by_id('txtKW').send_keys(TEXT)
driver.find_element_by_id('btnSimpleQry').click()

# 等待載入，選擇最高法院（若有）
try:
    driver.find_element_by_xpath("//div[@class='panel-body']   \
    //li/a[contains(text(),'最高法院')]").click()
except NoSuchElementException:
    pass

## 網頁檢索 / 爬蟲

In [9]:
# 建立須被彈性設定之變數
TIME = '109'
data = list()
status = True

# 由於定位資料於 iframe 內
# 因此需要告知瀏覽器切換到 iframe 嵌入框才能進行定位
iframe = driver.find_element_by_xpath("//iframe")
driver.switch_to.frame(iframe)

while(status):
    # 定位該頁所有搜尋結果之資訊，並進行迴圈檢視
    for position,each_date in enumerate(driver.find_elements_by_xpath("//table//tr[position() > 1 and not(contains(@class,'summary'))]/td[3]")):
        if each_date.text[:len(TIME)] == TIME:
            xpath_to_data = "(//table//tr[position() > 1 and not(contains(@class,'summary'))]/td[{}])[position() = {}]"
            data.append({
                'title'   : driver.find_element_by_xpath(xpath_to_data.format(2,position+1)).text,
                'date'    : driver.find_element_by_xpath(xpath_to_data.format(3,position+1)).text,
                'judge'   : driver.find_element_by_xpath(xpath_to_data.format(4,position+1)).text,
                'content' : '',
                'link'    : driver.find_element_by_xpath(f'{xpath_to_data.format(2,position+1)}//a').get_attribute("href")
            })  
    next_ = driver.find_elements_by_id('hlNext')
    if len(next_) != 0:
        next_[0].click()
    else:
        status = False

## 指定網頁判決內容截取

In [10]:
for each in data:
    driver = webdriver.Chrome(options=options)
    driver.get(each['link'])
    each['content'] = driver.find_element_by_xpath("(//div[@class='text-pre text-pre-in'])").text
    driver.close()
    sleep(1)

## 資料輸出

In [12]:
data

[{'title': '最高法院 108 年度 台上 字第 2708 號刑事判決（15K）',
  'date': '109.09.30',
  'judge': '詐欺',
  'content': '最高法院刑事判決\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000 108年度台上字第2708號\n上\u3000訴\u3000人\u3000林鶴爵\n\n\n選任辯護人\u3000顏世翠律師\n\u3000\u3000\u3000\u3000\u3000\u3000賴淳良律師\n\u3000\u3000\u3000\u3000\u3000\u3000蕭仰歸律師\n上列上訴人因詐欺案件，不服臺灣高等法院中華民國107 年10月\n23日第二審判決（107 年度上易字第1155號，起訴案號：臺灣臺\n北地方檢察署104 年度調偵續字第79號），提起上訴，本院判決\n如下：\n    主  文\n原判決關於林鶴爵犯罪所得沒收部分撤銷，發回臺灣高等法院。\n其他上訴駁回。\n    理  由\n壹、撤銷發回（即沒收上訴人林鶴爵犯罪所得）部分\n一、本件原判決撤銷第一審關於諭知上訴人無罪部分之判決，改\n    判除論處上訴人犯修正前詐欺取財罪刑（詳後述）外，另諭\n    知對上訴人未扣案之犯罪所得沒收（帳戶帳號及金額詳起訴\n    書），於全部或一部不能沒收或不宜執行沒收時，追徵其價\n    額。固非無見。\n二、惟查：\n（一）刑法沒收新制業將沒收定位為刑罰及保安處分以外獨立的\n      法律效果，已非從刑，不必從屬於主刑。是沒收雖以犯罪\n      （違法）行為為前提，但於不生裁判歧異之前提下，若罪\n      刑論科均無不合，僅沒收部分違法或不當，第三審自可僅\n      就沒收部分撤銷，另就罪刑部分判決駁回。\n（二）沒收新制在實體規範上擴大沒收之主體範圍，除沒收犯罪\n      行為人之財產外，新增剝奪犯罪行為人以外之第三人之財\n      產之實體規定；在刑事沒收程序方面，亦相應於刑事訴訟\n      法第7編之2增訂「沒收特別程序」編，賦予刑事訴訟程序\n      進行結果，財產可能被沒收之第三人程序主體地位，確保\n      其參與程序及尋求救濟之權利與機會。而沒收犯罪所得

## 資料整理

In [65]:
colname = []
df = pd.DataFrame.from_dict(data)

In [66]:
df = df.drop('content', axis=1)

In [67]:
title = pd.Series(df['title'])

In [58]:
df1 = title.str.split(' ', expand=True)
df2 = df1[6].str.split('號', expand=True)
df2 = df2[1].str.split('（', expand=True ).drop(1, axis=1)

In [59]:
df1 = df1.drop([0, 2, 4, 6], axis=1)
df = df.drop(['title'], axis=1)

In [61]:
df_final = pd.concat([df1, df2, df], axis=1)

In [63]:
df_final.columns = ['年度', '字號', '案號', '類型', '裁判日期', '裁判案由', '連結']

In [64]:
df_final

年度  字號    案號    類型       裁判日期              裁判案由  \
0  108  台上  2708  刑事判決  109.09.30                詐欺   
1  109  台上  1719  刑事判決  109.04.15         違反證券交易法等罪   
2  108  台上  4056  刑事判決  109.04.15         違反證券交易法等罪   
3  108  台上    16  刑事判決  109.01.16         違反證券交易法等罪   
4  109  台抗    46  刑事裁定  109.01.09  違反證券交易法等罪不服再執行羈押   

                                                  連結  
0  https://law.judicial.gov.tw/FJUD/data.aspx?ro=...  
1  https://law.judicial.gov.tw/FJUD/data.aspx?ro=...  
2  https://law.judicial.gov.tw/FJUD/data.aspx?ro=...  
3  https://law.judicial.gov.tw/FJUD/data.aspx?ro=...  
4  https://law.judicial.gov.tw/FJUD/data.aspx?ro=...